In [62]:
import pandas as pd
import numpy as np

In [63]:
gameInput = pd.read_csv("nhl-game-data/game.csv").dropna()
goalieInput = pd.read_csv("nhl-game-data/game_goalie_stats.csv")
skaterStatsInput = pd.read_csv("nhl-game-data/game_skater_stats.csv")
teamsStatsInput = pd.read_csv("nhl-game-data/game_teams_stats.csv")
teamInfoInput = pd.read_csv("nhl-game-data/team_info.csv")

In [64]:
goalies = goalieInput.loc[:,['game_id','team_id','shots','saves']].groupby(['game_id','team_id']).sum().reset_index()
goalies['percentage'] = (goalies['saves'] / goalies['shots'])*100
print(type(goalies.loc[goalies['game_id'] == 2010020001]['percentage'].values[0]))

<class 'numpy.float64'>


In [65]:
goalies1 = pd.DataFrame(columns=['game_id', 'home_percentage', 'away_percentage'])
for index, row in gameInput.iterrows():
    if not goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['away_team_id'])].empty:
        awayPerc = goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['away_team_id'])]
    if not goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['home_team_id'])].empty:
        homePerc = goalies.loc[(goalies['game_id'] == row['game_id']) & (goalies['team_id'] == row['home_team_id'])]
    d = {'game_id': row['game_id'], 'home_percentage': homePerc['percentage'].values[0], 'away_percentage': awayPerc['percentage'].values[0]}
    goalies1.at[index, :] = d
goalies1

,game_id,home_percentage,away_percentage
0,2.01103e+09,88.4615,88.8889
1,2.01103e+09,91.1765,95
2,2.01103e+09,89.2857,87.0968
3,2.01103e+09,90.9091,92.8571
4,2.01103e+09,90,96.4286
...,...,...,...
11429,2.01803e+09,73.913,93.1034
11430,2.01803e+09,91.3043,91.8919
11431,2.01803e+09,90.4762,97.4359
11432,2.01803e+09,87.0968,96.5517


In [66]:
gameInput1 = gameInput.join(goalies1, lsuffix='', rsuffix='_goalie')


In [67]:
gameAwayValues = gameInput1.assign(away_percentage=pd.to_numeric(gameInput1['away_percentage'], errors='coerce')).loc[:,['away_team_id', 'away_goals', 'home_goals', 'away_percentage', 'season']].groupby(['away_team_id', 'season']).mean().reset_index()
gameHomeValues = gameInput1.assign(home_percentage=pd.to_numeric(gameInput1['home_percentage'], errors='coerce')).loc[:,['home_team_id', 'away_goals', 'home_goals', 'home_percentage', 'season']].groupby(['home_team_id', 'season']).mean().reset_index()
# gameAwayValues
gameHomeValues
# gameInput1.loc[:,['away_team_id', 'away_percentage', 'season' ]]

,home_team_id,season,away_goals,home_goals,home_percentage
0,1,20102011,2.463415,2.292683,89.830349
1,1,20112012,2.461538,2.846154,90.677731
2,1,20122013,2.458333,2.750000,89.264884
3,1,20132014,2.243902,2.560976,91.644768
4,1,20142015,2.195122,2.219512,93.141136
...,...,...,...,...,...
260,53,20142015,3.243902,1.902439,90.767640
261,53,20152016,2.756098,2.975610,90.677792
262,53,20162017,2.853659,2.536585,91.836638
263,53,20172018,2.878049,2.487805,91.487818


In [77]:
dataSet2 = pd.DataFrame(columns=['game_id', 'season', 'date_time', 'home_team_id', '1_scored', '1_agg', '1_perc', 'away_team_id', '2_scored', '2_agg', '2_perc'])
# print(gameAwayValues.loc[(gameAwayValues['away_team_id'] == '4') & (gameAwayValues['season'] == '20102011')])
gameAwayValues = gameAwayValues.dropna()
gameHomeValues = gameHomeValues.dropna()
for index, row in gameInput1.iterrows():
    season = row['season'] - 10001
    if not gameAwayValues.loc[(gameAwayValues['away_team_id'] == row['away_team_id']) & (gameAwayValues['season'] == season)].empty:
        if not gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['home_team_id']) & (gameHomeValues['season'] == season)].empty:
            awayTeam = gameAwayValues.loc[(gameAwayValues['away_team_id'] == row['away_team_id']) & (gameAwayValues['season'] == season)]
            homeTeam = gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['home_team_id']) & (gameHomeValues['season'] == season)]
#             print(homeTeam, awayTeam)
            d = {'game_id': row['game_id'], 'season': row['season'], 'date_time': row['date_time'], 'home_team_id': row['home_team_id'], '1_scored': homeTeam['home_goals'].values[0], '1_agg': homeTeam['away_goals'].values[0], '1_perc': homeTeam['home_percentage'].values[0], 'away_team_id': row['away_team_id'], '2_scored': awayTeam['away_goals'].values[0], '2_agg': awayTeam['home_goals'].values[0],'2_perc': awayTeam['away_percentage'].values[0]}
            dataSet2.at[index, :] = d

In [84]:
dataSet2.to_csv('.\datasets\dataset2.csv', encoding='utf-8', index=False)